# StuPa-Vortrag zu Mensa-Preisen

In [15]:
%load_ext sql
%sql sqlite:///db.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
%reload_ext sql

In [17]:
from pandas import DataFrame, read_sql
from sqlalchemy import create_engine
import datetime

engine = create_engine('sqlite:///db.sqlite', echo=False)

##  Gesamtinflation

Berechnung anhand Vergleich von den Durchschnittspreisen von zwei Monaten

In [18]:
from typing import List

# functions

def average_dish_price(start_date: datetime.date, end_date: datetime.date, restaurant=None, category=None):
    return read_sql(f"""
    SELECT name_de, avg(priceStudents) as priceStudents FROM dish WHERE {restaurant and f"restaurant = '{restaurant}' AND"} {category and f"category = '{category}' AND"} date(date) BETWEEN date('{start_date.strftime("%Y-%m-%d")}') AND date('{end_date.strftime("%Y-%m-%d")}') GROUP BY name_de
    """, con=engine)['priceStudents'].mean()


In [19]:
from pandas.tseries.offsets import MonthEnd
from pandas import date_range

month_dish_prices = []
default = 0
for beg in date_range('2023-01-01', '2023-10-31', freq='MS'):
    if not default:
        default = average_dish_price(beg, (beg + MonthEnd(1)), restaurant='mensa-academica-paderborn', category='dish')
    month_dish_prices.append((beg, c:= average_dish_price(beg, (beg + MonthEnd(1)), restaurant='mensa-academica-paderborn', category='dish'), ((c/default)-1)))

month_dish_prices

[(Timestamp('2023-01-01 00:00:00', freq='MS'), 3.3223140495867765, 0.0),
 (Timestamp('2023-02-01 00:00:00', freq='MS'),
  3.351941747572816,
  0.008917789692315381),
 (Timestamp('2023-03-01 00:00:00', freq='MS'),
  3.4720720720720726,
  0.045076419703285575),
 (Timestamp('2023-04-01 00:00:00', freq='MS'),
  3.3585000000000007,
  0.010891791044776555),
 (Timestamp('2023-05-01 00:00:00', freq='MS'),
  3.3706422018348623,
  0.01454653338810541),
 (Timestamp('2023-06-01 00:00:00', freq='MS'),
  3.3282828282828283,
  0.0017965726920952108),
 (Timestamp('2023-07-01 00:00:00', freq='MS'),
  3.4191304347826086,
  0.029141250270387387),
 (Timestamp('2023-08-01 00:00:00', freq='MS'),
  3.353483606557377,
  0.009381881575728057),
 (Timestamp('2023-09-01 00:00:00', freq='MS'),
  3.3173076923076925,
  -0.0015068886337541443),
 (Timestamp('2023-10-01 00:00:00', freq='MS'),
  3.328787878787879,
  0.0019485903814264738)]

## Preisveränderung

Auflistung aller Preise pro Gericht und die Zeitspanne, in der sie auftreten

In [20]:
%%sql

SELECT name_de, priceStudents, min(date) AS min_date, max(date) AS max_date, count(name_de) as occurence FROM dish WHERE restaurant = 'mensa-academica-paderborn' AND category = 'dish' GROUP BY name_de, priceStudents

 * sqlite:///db.sqlite
Done.


name_de,priceStudents,min_date,max_date,occurence
1/2 knusprig gebratenes Grillhähnchen mit Zitrone,4.5,2022-08-15,2023-10-18,13
"2 Weißwürste mit Kartoffelsalat, Laugenschleife und süßem Senf",4.6,2022-09-21,2022-09-21,1
Apfel-Gemüse-Curry auf Glasnudeln mit Zitronengras und Ingwer,3.8,2023-05-11,2023-05-11,1
Apfel-Gemüse-Curry auf Mi-Nudeln mit Zitronengras und Ingwer,3.8,2023-06-22,2023-08-29,2
Apfel-Gemüse-Curry auf Mienudeln mit Zitronengras und Ingwer,3.8,2023-05-11,2023-05-11,1
Apfel-Holunderblüten Eistee,0.8,2022-09-15,2022-09-15,1
Apfelbeignet mit Zimt-Zucker auf Vanillesauce,1.5,2022-10-28,2022-10-28,1
Apfelstrudel mit Vanillesauce,1.8,2023-01-25,2023-01-25,1
Aramäische Rote Linsen Suppe mit Reis,2.3,2022-09-20,2022-09-20,1
Aramäische Rote Linsen Suppe mit Reis,2.4,2023-02-02,2023-02-02,1


In [21]:
price_steps = DataFrame(_)

### Gericht mit den höchsten Unterschieden

In [22]:
%%sql

SELECT name_de, min(priceStudents), max(priceStudents), max(priceStudents)/min(priceStudents) AS difference FROM (SELECT name_de, priceStudents, min(date) AS min_date, max(date) AS max_date FROM dish WHERE restaurant = 'mensa-academica-paderborn' AND category = 'dish' GROUP BY name_de, priceStudents) GROUP BY name_de ORDER BY difference DESC

 * sqlite:///db.sqlite
Done.


name_de,min(priceStudents),max(priceStudents),difference
Bockwurst vom Schwein,1.5,3.8,2.533333333333333
Soja-Bolognese mit Spaghetti,1.7,3.8,2.235294117647059
Khoresh Bademjan Auberginenauflauf,2.2,4.2,1.909090909090909
Gemüse-Reis-Pfanne vietnamesisch mit Curry-Kokos-Sauce,2.2,3.8,1.727272727272727
Würzige Hackfleischröllchen vom Schwein in Pusztasauce,1.7,2.8,1.6470588235294117
Röstiecken mit Kräuterquark,1.7,2.8,1.6470588235294117
Lasagne von Sojabolognese,1.7,2.8,1.6470588235294117
"Tofu-Gulasch mit Paprika, Zwiebeln und Kartoffeln",2.2,3.6,1.6363636363636362
"Bowl mit Belugalinsen, mariniertem Rotkohl und Karottensticks",3.4,5.5,1.6176470588235294
Mediterranes Rindergulasch mit Oliven und Paprika,2.8,4.5,1.6071428571428572


In [23]:
price_diff = DataFrame(_)

In [26]:
relevant_price_diff = price_diff[price_diff['difference'] > 1]
relevant_price_diff

,name_de,min(priceStudents),max(priceStudents),difference
0,Bockwurst vom Schwein,1.5,3.8,2.533333
1,Soja-Bolognese mit Spaghetti,1.7,3.8,2.235294
2,Khoresh Bademjan Auberginenauflauf,2.2,4.2,1.909091
3,Gemüse-Reis-Pfanne vietnamesisch mit Curry-Kok...,2.2,3.8,1.727273
4,Würzige Hackfleischröllchen vom Schwein in Pus...,1.7,2.8,1.647059
...,...,...,...,...
135,Crunchy Chicken Burger mit Pommes,4.4,4.5,1.022727
136,"Jerk Chicken mit Paprika, Ananas und Reis",4.5,4.6,1.022222
137,Teriyakisticks im Sub mit Chinakohl und Sprossen,4.8,4.9,1.020833
138,Gobi Butter Masala Blumenkohlcurry mit Reis,4.8,4.9,1.020833


In [25]:
relevant_dishes = relevant_price_diff['name_de']

price_steps[price_steps['name_de'].isin(relevant_dishes)]


,name_de,priceStudents,min_date,max_date,occurence
8,Aramäische Rote Linsen Suppe mit Reis,2.3,2022-09-20,2022-09-20,1
9,Aramäische Rote Linsen Suppe mit Reis,2.4,2023-02-02,2023-02-02,1
12,"Asia Pane mit Weizenpatty, Gemüse und Wasabima...",3.8,2023-09-08,2023-09-08,1
13,"Asia Pane mit Weizenpatty, Gemüse und Wasabima...",4.5,2023-07-14,2023-07-14,1
14,"Asia Pane mit Weizenpatty, Gemüse und Wasabima...",4.6,2023-03-10,2023-08-21,3
...,...,...,...,...,...
700,Würzige Hackfleischröllchen vom Schwein in Pus...,1.7,2022-09-09,2022-09-09,1
701,Würzige Hackfleischröllchen vom Schwein in Pus...,2.8,2022-10-21,2023-10-16,5
710,Zwiebelsteak vom Schwein mit Rosmarinjus,3.2,2023-05-03,2023-05-03,1
711,Zwiebelsteak vom Schwein mit Rosmarinjus,4.5,2023-08-30,2023-08-30,1
